In [1]:
# Program to pickle historical data and determine SD of nse underlyings

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=3) # kavi TWS live
ib=IB().connect('127.0.0.1', 4001, clientId=3) # kavi IBG live

In [6]:
import pandas as pd
import numpy as np

#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan

In [7]:
#******          Symbols Master      *******
#___________________________________________

symbols='ACC,ADANIENT,ADANIPORTS,ADANIPOWER,AJANTPHARM,ALBK,AMARAJABAT,AMBUJACEM,ANDHRABANK,APOLLOHOSP,\
APOLLOTYRE,ARVIND,ASHOKLEY,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,\
BALKRISIND,BALRAMCHIN,BANKBARODA,BANKINDIA,BATAINDIA,BEML,BERGEPAINT,BEL,BHARATFIN,BHARATFORG,\
BPCL,BHARTIARTL,INFRATEL,BHEL,BIOCON,BOSCHLTD,BRITANNIA,CADILAHC,CANFINHOME,CANBK,CAPF,CASTROLIND,\
CEATLTD,CENTURYTEX,CESC,CGPOWER,CHENNPETRO,CHOLAFIN,CIPLA,COALINDIA,COLPAL,CONCOR,CUMMINSIND,DABUR,\
DALMIABHA,DCBBANK,DHFL,DISHTV,DIVISLAB,DLF,DRREDDY,EICHERMOT,ENGINERSIN,EQUITAS,ESCORTS,EXIDEIND,\
FEDERALBNK,GAIL,GLENMARK,GMRINFRA,GODFRYPHLP,GODREJCP,GODREJIND,GRANULES,GRASIM,GSFC,HAVELLS,HCLTECH,\
HDFCBANK,HDFC,HEROMOTOCO,HEXAWARE,HINDALCO,HCC,HINDPETRO,HINDUNILVR,HINDZINC,ICICIBANK,ICICIPRULI,IDBI,\
IDEA,IDFCBANK,IDFC,IFCI,IBULHSGFIN,INDIANB,IOC,IGL,INDUSINDBK,INFIBEAM,INFY,INDIGO,IRB,ITC,JISLJALEQS,\
JPASSOCIAT,JETAIRWAYS,JINDALSTEL,JSWSTEEL,JUBLFOOD,JUSTDIAL,KAJARIACER,KTKBANK,KSCL,KOTAKBANK,KPIT,\
L%26TFH,LT,LICHSGFIN,LUPIN,M%26MFIN,MGL,M%26M,MANAPPURAM,MRPL,MARICO,MARUTI,MFSL,MINDTREE,MOTHERSUMI,\
MRF,MCX,MUTHOOTFIN,NATIONALUM,NBCC,NCC,NESTLEIND,NHPC,NIITTECH,NMDC,NTPC,ONGC,OIL,OFSS,ORIENTBANK,PAGEIND,\
PCJEWELLER,PETRONET,PIDILITIND,PEL,PFC,POWERGRID,PTC,PNB,PVR,RAYMOND,RBLBANK,RELCAPITAL,RCOM,RELIANCE,\
RELINFRA,RPOWER,REPCOHOME,RECLTD,SHREECEM,SRTRANSFIN,SIEMENS,SREINFRA,SRF,SBIN,SAIL,STAR,SUNPHARMA,SUNTV,\
SUZLON,SYNDIBANK,TATACHEM,TATACOMM,TCS,TATAELXSI,TATAGLOBAL,TATAMTRDVR,TATAMOTORS,TATAPOWER,TATASTEEL,TECHM,\
INDIACEM,RAMCOCEM,SOUTHBANK,TITAN,TORNTPHARM,TORNTPOWER,TV18BRDCST,TVSMOTOR,UJJIVAN,ULTRACEMCO,UNIONBANK,\
UBL,MCDOWELL-N,UPL,VEDL,VGUARD,VOLTAS,WIPRO,WOCKPHARMA,YESBANK,ZEEL'.split(',')

# Rectified for IBKR
isymbols = [i[0:9] for i in symbols]
isymbols = [s.replace('%26','') for s in isymbols]

In [113]:
#***   Determine Standard Deviation    ***
#_________________________________________

market = 'NSE'

# qualify contracts
contracts = [Stock(symbol=s, exchange=market) for s in isymbols[:3]]
qual_stocks = ib.qualifyContracts(*contracts)

# make a historical dictionary
hist_dict = dict([(c.symbol, ib.reqHistoricalData(contract=c, endDateTime='', durationStr='365 D', 
                                    barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)) for c in contracts])

data = [v for k, v in hist_dict.items()]

sd252 = [catch(lambda: np.std([i.close for i in data[j]][-252:], ddof=1)) 
         for j, k in enumerate(data)] # 252 trading days in a year

sd252_dict = dict(zip(isymbols, sd252))

df = pd.DataFrame.from_dict({(k, i.date): 
                        (i.open, i.high, i.low, i.close, i.average, i.volume) 
                        for k, v in hist_dict.items() for i in v}).T

df = df.rename(index=str, 
               columns={0: 'open', 1: 'high', 2: 'low', 3: 'close', 4: 'average', 5: 'volume'})

df_sd252 = pd.DataFrame.from_dict(sd252_dict, orient='index', columns=['sd252'])

symbols = df.index.get_level_values(level=0)

df['sd252'] = df_sd252.loc[symbols].values

df = df.reset_index().rename(columns=
                             {'level_0': 'Symbol', 'level_1': 'Date'}).set_index(['Symbol', 'Date'])

In [120]:
df.groupby(level='Symbol').close.rolling(2).std()

Symbol     Symbol     Date               
ACC        ACC        2017-04-19 00:00:00          NaN
                      2017-04-20 00:00:00     0.212132
                      2017-04-21 00:00:00    11.030866
                      2017-04-24 00:00:00    78.877761
                      2017-04-25 00:00:00     0.212132
                      2017-04-26 00:00:00     4.030509
                      2017-04-27 00:00:00    24.324473
                      2017-04-28 00:00:00     6.682159
                      2017-05-02 00:00:00    23.688077
                      2017-05-03 00:00:00    10.960155
                      2017-05-04 00:00:00    10.960155
                      2017-05-05 00:00:00    25.526555
                      2017-05-08 00:00:00    36.133157
                      2017-05-09 00:00:00     7.106423
                      2017-05-10 00:00:00     7.672109
                      2017-05-11 00:00:00     0.742462
                      2017-05-12 00:00:00     6.682159
                      2